<a href="https://colab.research.google.com/github/navneet-g/sde-skills-genai-workshop/blob/main/Workshop_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-reqs

1. Bring a laptop or pair with someone to participate in hands-on lab
2. Signup for free google Colab at https://colab.research.google.com/
3. Get an api key at https://platform.openai.com/api-keys
4. We will follow the Notebook at https://github.com/navneet-g/sde-skills-genai-workshop/blob/main/Workshop_1.ipynb




# Guess the word


### Example 1
* He loved to study and when he grew up he became a <?>

### Example 2
* He wore a white coat and stethoscope to work everyday. He loved to study and when he grew up he became a <?>.




That is how transformers work, they guess the word based on the context.

* Large langugage models are trained on lots of data and can help and

In [1]:
!pip install --q -U langchain langchain_community \
openai langchain-openai selenium unstructured \
langchain-text-splitters unstructured faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_community.document_loaders import SeleniumURLLoader  # loading documents
from langchain.text_splitter import CharacterTextSplitter  # splitting text
from langchain_community.vectorstores import (
    FAISS,
)  # creating vector store from embeddings; can use chromadb instead as well
from langchain.chains import RetrievalQA  # creating qa system
from langchain_openai import ChatOpenAI  # using llm for qa system
from langchain_openai import OpenAIEmbeddings  # embedding text with openai

from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

import openai

import importlib
google_collab = importlib.util.find_spec('google')
local_run = google_collab is None

# try:
#     imp.find_module('google')
#     from google.colab import userdata
#     local_run = False
# except ImportError:
#     local_run = True

print("Running locally:", local_run)

Running locally: False


In [4]:
if local_run:
  llm = ChatOllama(model="llama3:8b")
  embeddings = OllamaEmbeddings(model="nomic-embed-text")
else:
  from google.colab import userdata
  openai_api_key=userdata.get('OPENAI_API_KEY')
  llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


# docs https://python.langchain.com/docs/integrations/llms/google_ai/


In [5]:
llm.invoke("When were the last academy awards").content

'The last Academy Awards, also known as the Oscars, were held on April 25, 2021.'

# **In context learning**

In [6]:
context = "Context: \n\
The 96th Academy Awards ceremony, which was presented by the Academy of Motion Picture Arts and Sciences (AMPAS), took place on March 10, 2024, at the Dolby Theatre in Hollywood, Los Angeles.[6] During the gala, the AMPAS presented Academy Awards (commonly referred to as Oscars) in 23 categories honoring films released in 2023. Comedian Jimmy Kimmel hosted the show for the fourth time.[a] \n"

question = "When were the 96th academy awards?"

llm.invoke(context + question).content


'The 96th Academy Awards took place on March 10, 2024.'

In [7]:
question ="Who were the academy awards nominees"
llm.invoke(context + question).content

"I'm sorry, but I do not have access to real-time information, so I cannot provide you with the specific nominees for the 96th Academy Awards in 2024. I recommend checking the official Academy Awards website or a reliable news source for the most up-to-date information on the nominees."

# **RAG**

In [8]:
# load url
urls = [
    "https://en.wikipedia.org/wiki/96th_Academy_Awards",
]

loader = SeleniumURLLoader(urls=urls)
data = loader.load()


# split document by character
print("Splitting document by character...")
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=1000, chunk_overlap=200
)

# split into multiple documents
print("Splitting into multiple documents...")
docs = text_splitter.split_documents(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Splitting document by character...
Splitting into multiple documents...


In [9]:
print("Creating vector store...")
# create vector store
db = FAISS.from_documents(docs, embedding=embeddings)

Creating vector store...


In [10]:

# create retriever to ask questions using openai and vector store
print("Creating retriever...")
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 10}),
)

Creating retriever...


In [11]:
def ask_question(question):
    print("Query: " + question)
    print("Response: " + qa.invoke(question)['result'])
    print("===============")

In [12]:
ask_question("Who were the academy awards winners?")
ask_question("What date did the academy awards happen?")
ask_question("What date did the 96th academy awards happen?")

Query: Who were the academy awards winners?
Response: The winners of the 96th Academy Awards included Oppenheimer, which won 7 awards, Poor Things, which won 4 awards, and The Zone of Interest, which won 2 awards. Specific categories and winners can be found in the full list of winners released by the Academy of Motion Picture Arts and Sciences.
Query: What date did the academy awards happen?
Response: The 96th Academy Awards ceremony took place on March 10, 2024, at the Dolby Theatre in Hollywood, Los Angeles.
Query: What date did the 96th academy awards happen?
Response: The 96th Academy Awards ceremony took place on March 10, 2024.


## **Tools and Agents**

Ref: [Building LLM Agents in 3 Levels of Complexity: From Scratch, OpenAI Functions & LangChain](https://www.youtube.com/watch?v=v1tyQtncsE4)

In [17]:
import subprocess

class ModelWithTools:
    def __init__(self, llm):
        self.llm = llm

    def get_response(self, prompt_question):
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                    {"role": "user", "content": prompt_question}]
        return llm.invoke(messages).content

    def create_directory(self, directory_name):
        subprocess.run(["mkdir", directory_name])

    def create_file(self, file_name):
        subprocess.run(["touch", file_name])

    def list_files(self):
        subprocess.run(["ls"])

    def execute_function_call(self, function_call_string: str):
        exec(function_call_string)

In [18]:
model = ModelWithTools(llm=llm)

In [19]:
from IPython.display import Markdown

task_description = "Create a folder called 'sde-skills-workshop'. Inside that folder, create a file called 'llm-agent-demo.md"
output = model.get_response(f"""Given a task that will be fed as input, and consider you have access to the following functions:

    def create_directory(self, directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])

    def create_file(self, file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])

    def list_files(self):
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])

    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    For example:

    task: 'create a folder named lucas-the-agent-master'
    output: model.create_directory('lucas-the-agent-master')

    task: 'create a file named the-10-master-rules.md'
    output: model.create_file('the-10-master-rules.md')

    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    task: {task_description}
    output:\n
    """)

print(output)

model.create_directory('sde-skills-workshop')
model.create_file('sde-skills-workshop/llm-agent-demo.md')


In [20]:
model.execute_function_call(output)

In [ ]:
!ls

Another copy of building-agents.ipynb Workshop-1.ipynb
Copy of building-agents.ipynb         sde-skills-workdshop
LangChain_RAG_GoogleAI.ipynb          sde-skills-workshop
LangChain_RAG_OpenAI.ipynb
